In [1]:
import os,sys, json,re, pickle
import magic, hashlib,  traceback ,ntpath, collections ,lief
from capstone import *
from capstone.x86 import *
import torch.nn as nn
import lief,traceback
from elftools.elf.elffile import ELFFile
from transformers import AdamW,AutoTokenizer
from tqdm import tqdm  # for our progress bar
from sklearn.metrics import precision_recall_fscore_support , accuracy_score,f1_score, confusion_matrix,mean_squared_error, mean_absolute_error, r2_score
from numpy import *
from num2words import num2words
import pandas as pd

In [2]:
import clang.cindex
from clang import cindex
from clang.cindex import CursorKind

In [3]:

# Importing required module
import subprocess
import concurrent.futures
from os import walk
import os, signal
from os import listdir
from os.path import isfile, join
import  os
import multiprocessing
import hashlib

In [4]:
def hash_file(filepath, algo='sha256'):
    try:
        hasher = getattr(hashlib, algo)()
        with open(filepath, 'rb') as f:
            while chunk := f.read(8192):  # Read in 8KB chunks
                hasher.update(chunk)
        return hasher.hexdigest()
    except FileNotFoundError:
        print(f"[!] File not found: {filepath}")
    except AttributeError:
        print(f"[!] Invalid hash algorithm: {algo}")

In [5]:

import clang.cindex
from clang.cindex import CursorKind, TypeKind



def find_constructs(source_path ,shared_dict):

    try:


        index = clang.cindex.Index.create()
        tu = index.parse(source_path)#, args=['-std=c11', '-I/path/to/includes'])

        switches = []
        funcptr_decls = []
        funcptr_params = []
        nested_loops = []

        loop_kinds = {
            CursorKind.FOR_STMT,
            CursorKind.WHILE_STMT,
            CursorKind.DO_STMT
        }

        def is_loop(node):
            return node.kind in loop_kinds

        def contains_loop(node):
            for child in node.get_children():
                if is_loop(child):
                    return True
                if contains_loop(child):
                    return True
            return False

        def visit(node):
            if node.location.file and node.location.file.name != source_path:
                return
            # --- Switch statements ---
            if node.kind == CursorKind.SWITCH_STMT:
                switches.append((node.location.line, "switch"))

            # --- Function pointer declarations ---
            if node.kind == CursorKind.VAR_DECL:
                if node.type.kind == TypeKind.POINTER:
                    pointee = node.type.get_pointee()
                    if pointee.kind == TypeKind.FUNCTIONPROTO:
                        funcptr_decls.append((node.spelling, node.location.line))

            # --- Function declarations with function pointer parameters ---
            if node.kind == CursorKind.FUNCTION_DECL:
                for arg in node.get_arguments():
                    if arg.type.kind == TypeKind.POINTER:
                        pointee = arg.type.get_pointee()
                        if pointee.kind == TypeKind.FUNCTIONPROTO:
                            funcptr_params.append((node.spelling, arg.spelling, node.location.line))

            # --- Nested loop detection ---
            if is_loop(node) and contains_loop(node):
                nested_loops.append((node.kind.name, node.location.line))

            for child in node.get_children():
                visit(child)

        visit(tu.cursor)

        result = {
            "switch_statements": len(switches),
            "funcptr_declarations": len(funcptr_decls),
            "funcptr_parameters": len(funcptr_params),
            "nested_loops": len(nested_loops)
        }

        result_detail = {
            "switch_statements": switches,
            "funcptr_declarations": funcptr_decls,
            "funcptr_parameters": funcptr_params,
            "nested_loops": nested_loops
        }
        src_file_hash = hash_file(source_path)
        shared_dict[src_file_hash] = result
        return result

    except Exception as e:
        traceback.print_exc()
        return None


In [6]:

root    ='/media/raisul/clones_100k'
import pickle
all_c_paths = []
all_make_dir_paths = []
with open('/home/raisul/pun_dataset/dev/c_files_n_projs.ignore.pkl', 'rb') as file:
    all_c_paths,all_make_dir_paths  = pickle.load(file)  

all_c_paths= all_c_paths#[0:10]
all_c_paths = [ f_path.replace('/media/raisul/' , '/media/raisul/nahid_personal/') for f_path in all_c_paths]

In [7]:
def compute_category_totals(shared_dict):
    totals = {
        "switch_statements": 0,
        "funcptr_declarations": 0,
        "funcptr_parameters": 0,
        "nested_loops": 0
    }
    totals_files =  {
        "switch_statements": 0,
        "funcptr_declarations": 0,
        "funcptr_parameters": 0,
        "nested_loops": 0
    }

    for result in shared_dict.values():
        for key in totals:
            value = result.get(key, 0)
            totals[key] += value

            if value>0:
                totals_files[key] +=1
             

    return totals ,totals_files

In [8]:
import multiprocessing
from multiprocessing import Manager

if __name__ == "__main__":

    print(f"There are {multiprocessing.cpu_count()} CPUs available")
    number_processes = multiprocessing.cpu_count()

    with Manager() as manager:
        shared_dict = manager.dict()

        pool = multiprocessing.Pool(processes=number_processes)

        for path in all_c_paths:
            pool.apply_async(find_constructs, args=(path, shared_dict))

        pool.close()
        pool.join()

        # ✅ COPY the contents to a regular dict BEFORE manager shuts down
        final_dict = dict(shared_dict)



There are 192 CPUs available


Traceback (most recent call last):
  File "/tmp/ipykernel_41513/1498684542.py", line 12, in find_constructs
    tu = index.parse(source_path)#, args=['-std=c11', '-I/path/to/includes'])
         ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/raisul/anaconda3/envs/pytorch/lib/python3.12/site-packages/clang/cindex.py", line 2899, in parse
    return TranslationUnit.from_source(path, args, unsaved_files, options, self)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/raisul/anaconda3/envs/pytorch/lib/python3.12/site-packages/clang/cindex.py", line 3019, in from_source
    raise TranslationUnitLoadError("Error parsing translation unit.")
clang.cindex.TranslationUnitLoadError: Error parsing translation unit.
Traceback (most recent call last):
  File "/tmp/ipykernel_41513/1498684542.py", line 12, in find_constructs
    tu = index.parse(source_path)#, args=['-std=c11', '-I/path/to/includes'])
         ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/raisul/anaconda

In [9]:
#jupyter nbconvert --to script data_pipe.ipynb
# accelerate launch data_pipe.py > log.txt

In [10]:
totals,totals_files = compute_category_totals(final_dict)
for k, v in totals.items():
    print(f"{k}: {v}")
print('\n\n')
for k, v in totals_files.items():
    print(f"{k}: {v}")

switch_statements: 368380
funcptr_declarations: 15225
funcptr_parameters: 51882
nested_loops: 955095



switch_statements: 147375
funcptr_declarations: 6438
funcptr_parameters: 20101
nested_loops: 355416


In [11]:
len(final_dict.items())

1630053

In [ ]:
# switch_statements: 368380
# funcptr_declarations: 15225
# funcptr_parameters: 51882
# nested_loops: 955095



# switch_statements: 147375
# funcptr_declarations: 6438
# funcptr_parameters: 20101
# nested_loops: 355416